# EE954 : Deep Learning Fundamentals
### Assignment 1

**Group Number:** 12  
**Team Members:**  
- Lokesh  (Roll No: 241562482)  
- Akshay (Roll No: _____)

---



General Instructions 

• Late submissions will not be accepted. 

• Any form of plagiarism will result inpenalties. If you refer to any online material or books,cite them properly.

• You may use Google Colab or Kaggle to train your models. 

• The use of the Numpy library is permitted. 

• The use of Tensor Flow library is strictly prohibited. 

• The use of PyTorch library is allowed with restrictions.

### 1. DatasetPreparation (5Marks) 

1.1 Download and Split (2Marks) 

•Download the Fashion-MNIST dataset. You can either download it from here,or import it directly into your code using PyTorch’s torchvision.datasets module.

• Both sources provide separate training and test splits; however, you will need to create a separate validation set from the training data. 

From Perplexity.AI - 
'

Fashion-MNIST is a widely used machine learning dataset consisting of 70,000 grayscale images (28x28 pixels) of fashion items from 10 categories, such as T-shirts, trousers, dresses, and shoes. There are 60,000 images for training and 10,000 for testing. Each image is labeled with one of the 10 clothing classes. Fashion-MNIST was designed as a more challenging, modern replacement for the original MNIST handwritten digits dataset, while maintaining the same format and structure for easy benchmarking and comparison of machine learning algorithms.

Each example is a 28x28 grayscale image of a fashion item, labeled with one of 10 classes:
0: T-shirt/top
1: Trouser
2: Pullover
3: Dress
4: Coat
5: Sandal
6: Shirt
7: Sneaker
8: Bag
9: Ankle boot

'

Documentation on dataset - https://github.com/zalandoresearch/fashion-mnist

In [5]:
!pip install torch torchvision
import torch
from torchvision import datasets, transforms

# Download Fashion-MNIST training set
#dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())  # Note : Since we are using transforms.ToTensor(), the images will be converted to PyTorch tensors and normalized to [0, 1]. This is relevant for section 1.2 which calls for an explicit normalization step.normalization function for completeness., even though the images have already been normalized to [0, 1] by the transform. from 0 to 255 to 0 to 1.


# Define the augmentation pipeline
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # 50% chance to flip horizontally[3]
    transforms.RandomVerticalFlip(p=0.5),    # 50% chance to flip vertically[4][2]
    transforms.RandomRotation(degrees=360),  # Random rotation by any angle between 0 and 360 degrees[1][6]
    transforms.ToTensor()                    # Convert image to tensor and normalize to [0, 1]
])

# Apply the transform when loading the dataset
dataset = datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True, 
    transform=transform
)

# Split into training and validation sets (e.g., 80% train, 20% val)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


In [6]:
# For datasets created via random_split
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")

# For a single sample (image, label)
image, label = train_dataset[0]
print(f"Image shape: {image.shape}, Label: {label}")


# Helper function to extract all labels from a Subset
def get_all_labels(subset):
    return [subset[i][1] for i in range(len(subset))]

# Get all labels
train_labels = get_all_labels(train_dataset)
val_labels = get_all_labels(val_dataset)

# Get unique labels
unique_train_labels = set(train_labels)
unique_val_labels = set(val_labels)

print("Unique labels in train_dataset:", unique_train_labels)
print("Unique labels in val_dataset:", unique_val_labels)

all_pixels = torch.cat([train_dataset[i][0].view(-1) for i in range(len(train_dataset))])
min_pixel = all_pixels.min().item()
max_pixel = all_pixels.max().item()
print("Min pixel value:", min_pixel)
print("Max pixel value:", max_pixel)

Training set size: 48000
Validation set size: 12000
Image shape: torch.Size([1, 28, 28]), Label: 3
Unique labels in train_dataset: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Unique labels in val_dataset: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
Min pixel value: 0.0
Max pixel value: 1.0


1.2 Data Preprocessing (3 Marks) 

• Define a preprocess function to preprocess the images. (The function should have the same name).
• The function should flatten the images. 
• The function should also normalize the pixel values to the range [0,1]. Depending on how you have implemented the code so far, the pixel values might already be normalized to this range. However, for clarity and completeness, include an explicit normalization step regardless.

In [7]:
from torch.utils.data import Dataset

def preprocess(image):
    """
    Flattens a 28x28 image to a 784-dim vector and normalizes pixel values to [0, 1].
    Args:
        image (torch.Tensor): Image tensor of shape (1, 28, 28) or (28, 28)
    Returns:
        torch.Tensor: Flattened and normalized image of shape (784,)
    Note that if the image is already normalized to the range [0, 1] when we used the tranform.totensor() while calling the dataset, this function will not change it.
    """
    # Ensure image is float and normalize to [0,1]
    image = image.float() / 255.0 if image.max() > 1 else image.float()
    # Flatten the image
    return image.view(-1)

# Create a wrapper dataset to apply preprocessing

class PreprocessedDataset(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        return preprocess(image), label  # Apply preprocessing

# Wrap your existing datasets
train_dataset_preprocessed = PreprocessedDataset(train_dataset)
val_dataset_preprocessed = PreprocessedDataset(val_dataset)

# Verify
sample_img, sample_label = train_dataset_preprocessed[1]
print(sample_img.shape)          # torch.Size([784])
print(sample_img.min(), sample_img.max())  # Should be tensor(0.) tensor(1.)
print(sample_label)            # Corresponding label
print("Unique labels in train_dataset_preprocessed:", set(get_all_labels(train_dataset_preprocessed)))

torch.Size([784])
tensor(0.) tensor(1.)
5
Unique labels in train_dataset_preprocessed: {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
